# 如何更新文件到quant-wiki
## 更新基本概念时的流程：
- 翻译文章
- 将文章分类放进对应的文件夹下面（llm进行分类）
- 更新知识框架部分
- 将文章进一步分类，更新mkdocs.yml
- 根据分类结果更新概述

## 翻译所用的脚本

In [ ]:
import os
import glob
import json
import argparse
import threading
import concurrent.futures
import re
import requests
import time
import random
from typing import List, Dict, Any, Tuple, Callable

class MarkdownTranslator:
    def __init__(self, api_key: str, base_url: str, model: str, max_tokens: int = 4000, max_retries: int = 3):
        self.api_key = api_key
        self.base_url = base_url
        self.model = model
        self.max_tokens = max_tokens
        self.max_retries = max_retries
        self.lock = threading.Lock()
        
    def _retry_with_backoff(self, func: Callable, *args, **kwargs) -> Any:
        """使用指数退避策略重试函数调用"""
        retry_count = 0
        while retry_count <= self.max_retries:
            try:
                return func(*args, **kwargs)
            except Exception as e:
                retry_count += 1
                if retry_count > self.max_retries:
                    # 已达到最大重试次数，重新抛出异常
                    raise e
                
                # 计算退避时间（指数增长，有随机抖动）
                backoff_time = min(2 ** (retry_count - 1), 60) + random.uniform(0, 1)
                
                with self.lock:
                    print(f"API调用失败: {str(e)}. 将在 {backoff_time:.2f} 秒后进行第 {retry_count}/{self.max_retries} 次重试...")
                
                time.sleep(backoff_time)
        
        # 这里不应该执行到，但为了逻辑完整性添加
        raise Exception("所有重试尝试均失败")
        
    def _translate_text_impl(self, text: str) -> str:
        """实际执行翻译的内部函数"""
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": "你是一个专业的翻译助手，请将以下Markdown文本翻译成流畅、自然的中文，保留原始的Markdown格式和标记。代码块内容不需要翻译，引用文献部分也不需要翻译"},
                {"role": "user", "content": text}
            ],
            "temperature": 0.3
        }
        
        response = requests.post(f"{self.base_url}/v1/chat/completions", 
                                headers=headers, 
                                json=payload,
                                timeout=60)
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"]
        
    def translate_text(self, text: str) -> str:
        """调用LLM API翻译文本（带重试功能）"""
        try:
            return self._retry_with_backoff(self._translate_text_impl, text)
        except Exception as e:
            print(f"翻译文本最终失败: {str(e)}")
            return text  # 所有重试都失败时返回原文本
    
    def split_text(self, text: str) -> List[str]:
        """将长文本分割成不超过最大token数的片段"""
        # 使用简单的估算方法：1个中文字符约1.5个token，1个英文单词约1.3个token
        # 这只是一个粗略估计，实际token数会有所不同
        
        # 先保护代码块
        code_blocks = []
        
        def replace_code_blocks(match):
            code_blocks.append(match.group(0))
            return f"__CODE_BLOCK_{len(code_blocks)-1}__"
        
        # 使用正则表达式匹配代码块
        code_block_pattern = r"```[\s\S]*?```"
        text_with_placeholders = re.sub(code_block_pattern, replace_code_blocks, text)
        
        # 按段落分割文本
        paragraphs = re.split(r'\n\s*\n', text_with_placeholders)
        
        # 重新组合段落，确保每个片段不超过最大token数
        chunks = []
        current_chunk = ""
        current_tokens = 0
        
        for para in paragraphs:
            # 估算段落的token数
            estimated_tokens = len(para.split()) * 1.3 + len(re.findall(r'[\u4e00-\u9fff]', para)) * 1.5
            
            if current_tokens + estimated_tokens > self.max_tokens * 0.8:  # 留出20%的余量
                if current_chunk:
                    chunks.append(current_chunk)
                current_chunk = para
                current_tokens = estimated_tokens
            else:
                if current_chunk:
                    current_chunk += "\n\n" + para
                else:
                    current_chunk = para
                current_tokens += estimated_tokens
        
        if current_chunk:
            chunks.append(current_chunk)
            
        # 还原代码块
        final_chunks = []
        for chunk in chunks:
            for i, code_block in enumerate(code_blocks):
                chunk = chunk.replace(f"__CODE_BLOCK_{i}__", code_block)
            final_chunks.append(chunk)
            
        return final_chunks
    
    def _translate_filename_impl(self, filename: str) -> str:
        """实际执行文件名翻译的内部函数"""
        # 分离文件名和扩展名
        name, ext = os.path.splitext(filename)
        
        # 如果文件名看起来已经是中文，就不翻译了
        if re.search(r'[\u4e00-\u9fff]', name):
            return filename
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": "请将以下英文文件名（金融或经济领域）翻译成简短、清晰的中文。并以中文名_英文名的格式返回。只翻译内容含义，不要添加额外词语，保持简洁。"},
                {"role": "user", "content": name}
            ],
            "temperature": 0.3
        }
        
        response = requests.post(f"{self.base_url}/v1/chat/completions", 
                                headers=headers, 
                                json=payload,
                                timeout=30)
        response.raise_for_status()
        result = response.json()
        translated_name = result["choices"][0]["message"]["content"].strip()
        
        # 确保翻译后的文件名是有效的
        translated_name = re.sub(r'[\\/*?:"<>|]', '', translated_name)  # 移除Windows文件名中不允许的字符
        
        return translated_name + ext
    
    def translate_filename(self, filename: str) -> str:
        """翻译文件名为中文（带重试功能）"""
        try:
            return self._retry_with_backoff(self._translate_filename_impl, filename)
        except Exception as e:
            print(f"文件名翻译最终失败: {str(e)}")
            return filename  # 所有重试都失败时返回原文件名
    
    def translate_file(self, file_path: str, output_dir: str = None) -> None:
        """翻译单个文件"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
            # 分割文本
            chunks = self.split_text(content)
            
            # 翻译每个片段
            translated_chunks = []
            for i, chunk in enumerate(chunks):
                with self.lock:
                    print(f"正在翻译 {file_path} 的第 {i+1}/{len(chunks)} 部分...")
                translated_chunk = self.translate_text(chunk)
                translated_chunks.append(translated_chunk)
                
            # 合并翻译后的文本
            translated_content = "\n\n".join(translated_chunks)
            
            # 翻译并确定输出路径
            file_name = os.path.basename(file_path)
            translated_file_name = self.translate_filename(file_name)
            
            if output_dir:
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                output_path = os.path.join(output_dir, translated_file_name)
            else:
                dir_name = os.path.dirname(file_path)
                output_path = os.path.join(dir_name, translated_file_name)
            
            # 写入翻译后的文件
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(translated_content)
                
            with self.lock:
                print(f"✅ 已完成文件翻译: {file_path} -> {output_path}")
                
        except Exception as e:
            with self.lock:
                print(f"❌ 翻译文件失败 {file_path}: {str(e)}")
    
    def translate_directory(self, input_dir: str, output_dir: str = None, max_workers: int = 5) -> None:
        """使用多线程翻译文件夹内所有Markdown文件"""
        # 查找所有markdown文件
        md_files = glob.glob(os.path.join(input_dir, "**", "*.md"), recursive=True)
        
        if not md_files:
            print(f"在 {input_dir} 中未找到Markdown文件")
            return
        
        print(f"发现 {len(md_files)} 个Markdown文件，准备翻译...")
        
        # 使用线程池进行多线程翻译
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(self.translate_file, file_path, output_dir) for file_path in md_files]
            
            # 等待所有任务完成
            concurrent.futures.wait(futures)
            
        print(f"所有文件翻译完成！")


def main():
    parser = argparse.ArgumentParser(description="使用大语言模型将Markdown文件翻译成中文")
    parser.add_argument("--input_dir", "-i", required=True, help="包含Markdown文件的输入目录")
    parser.add_argument("--output_dir", "-o", help="翻译后文件的输出目录")
    parser.add_argument("--api_key", "-k", required=True, help="大语言模型API密钥")
    parser.add_argument("--base_url", "-u", default="https://api.openai.com", help="API基础URL")
    parser.add_argument("--model", "-m", default="gpt-3.5-turbo", help="使用的模型名称")
    parser.add_argument("--max_tokens", "-t", type=int, default=4000, help="每个翻译片段的最大token数")
    parser.add_argument("--threads", "-n", type=int, default=3, help="并行处理的线程数")
    parser.add_argument("--retries", "-r", type=int, default=3, help="API调用失败时的最大重试次数")
    
    args = parser.parse_args()
    
    translator = MarkdownTranslator(
        api_key=args.api_key,
        base_url=args.base_url,
        model=args.model,
        max_tokens=args.max_tokens,
        max_retries=args.retries
    )
    
    translator.translate_directory(
        input_dir=args.input_dir,
        output_dir=args.output_dir,
        max_workers=args.threads
    )


if __name__ == "__main__":
    main()


## 将文件分类放进对应的文件夹下面（手动）

## 更新知识框架部分

In [ ]:
import os
import glob

def generate_navigation(directory_path):
    # 获取指定目录下所有的md文件
    md_files = glob.glob(os.path.join(directory_path, "*.md"))
    
    # 生成导航列表
    navigation = []
    for file_path in md_files:
        # 获取文件名（不含扩展名）
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        # 获取相对路径，并移除 ../../../docs/basic/ 前缀
        relative_path = os.path.relpath(file_path, "/Users/fengwenjun/Desktop/CODE/Automatic tools for LLMQuant/quant-wiki/docs/basic")
        # 确保路径使用正斜杠
        relative_path = relative_path.replace('\\', '/')
        # 生成导航项
        nav_item = f"- [{file_name}]({relative_path})"
        navigation.append(nav_item)
    
    return "\n".join(sorted(navigation))

# 使用示例
directory_path = "/Users/fengwenjun/Desktop/CODE/Automatic tools for LLMQuant/quant-wiki/docs/basic/stat"
nav_text = generate_navigation(directory_path)
print(nav_text)

## 将文章进一步分类，更新mkdocs.yml(使用cursor)
先使用git show --name-only a1b2c  打印出文件变更情况
使用cursor直接编辑 mkdocs.yml：我更新了很多文件，请你根据此来分类并更新mkdocs中基本概念的部分

## 根据分类结果更新综述

将mkdocs中的内容输入大模型进行转写，用下面的prompt进行转写

你是一个文案编辑整理大师，你需要转写用户提供的文本为一个markdown。实例输入：
    - 金融术语: 
      - 市场与交易:
        - 一级市场: basic/finance/一级市场_Primary Market.md
        - 二级市场: basic/finance/二级市场_Secondary Market.md
        - 债券市场: basic/finance/债券市场_Bond Market.md
        - 外汇市场: basic/finance/外汇市场_Foreign Exchange.md
        - 股市: basic/finance/股市_Stock Market.md
        - 熊市: basic/finance/熊市_Bear Market.md
        - 牛市: basic/finance/牛市_Bull Market.md
        - 纳斯达克: basic/finance/纳斯达克_Nasdaq.md
      - 金融工具:
        - 股权: basic/finance/股权_Equity.md
        - 期货: basic/finance/期货_Futures.md 
输出：
# 金融术语

## 章节导航

### 市场与交易

- [一级市场](一级市场_Primary Market.md)
- [二级市场](二级市场_Secondary Market.md)
- [债券市场](债券市场_Bond Market.md)
- [外汇市场](外汇市场_Foreign Exchange.md)
- [股市](股市_Stock Market.md)
- [熊市](熊市_Bear Market.md)
- [牛市](牛市_Bull Market.md)
- [纳斯达克](纳斯达克_Nasdaq.md)

### 金融工具

- [股权](股权_Equity.md)
- [期货](期货_Futures.md)

In [3]:
from openai import OpenAI

def send_messages(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message

client = OpenAI(
    api_key="sk-258b756def1b41dab057106a0998f1ff",
    base_url="https://api.deepseek.com",
)

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

messages = [{"role": "user", "content": "How's the weather in Hangzhou?"}]
message = send_messages(messages)
print(f"User>\t {messages[0]['content']}")

tool = message.tool_calls[0]
messages.append(message)

messages.append({"role": "tool", "tool_call_id": tool.id, "content": "24℃"})
message = send_messages(messages)
print(f"Model>\t {message.content}")

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'